In [ ]:
import numpy as np
import dautil as dl
import ch7util
from scipy.signal import argrelmin
from scipy.signal import argrelmax
import matplotlib.pyplot as plt

In [ ]:
context = dl.nb.Context('calmar_sortino')
lr = dl.nb.LatexRenderer(chapter=7, start=4, context=context)
lr.render(r'S = \frac{R-T}{DR}')

In [ ]:
def calc_sortino(rets):
    # Returns below target
    semi_var = rets[rets < 0] ** 2
    semi_var = semi_var.sum()/len(rets)
    sortino = np.sqrt(semi_var)

    return rets.mean()/sortino

In [ ]:
def calc_calmar(rets):
    # Peaks and bottoms indexes in sequence
    mins = np.ravel(argrelmin(rets))
    maxs = np.ravel(argrelmax(rets))
    extrema = np.concatenate((mins, maxs))
    extrema.sort()

    return -rets.mean()/np.diff(rets[extrema]).min()

In [ ]:
ohlc = dl.data.OHLC()
dfb = dl.report.DFBuilder(cols=['Ticker', 'Sortino', 'Calmar'])

for symbol in ch7util.STOCKS:
    stock = ohlc.get(symbol)
    rets = ch7util.log_rets(stock['Adj Close'])
    sortino = calc_sortino(rets)
    calmar = calc_calmar(rets)
    dfb.row([symbol, sortino, calmar])

df = dfb.build(index=ch7util.STOCKS).dropna()

In [ ]:
%matplotlib inline
dl.options.mimic_seaborn()
_, ax = plt.subplots()
ax.scatter(df['Sortino'], df['Calmar'])
dl.plotting.plot_polyfit(ax, df['Sortino'], df['Calmar'])
dl.plotting.plot_text(ax, df['Sortino'], df['Calmar'], ch7util.STOCKS)
ax.set_xlabel('Sortino')
ax.set_ylabel('Calmar')
ax.set_title('Sortino & Calmar Ratios')